# Citation Validation Agentic AI for Google Capstone Project
- Interface: Google Docs + Google Apps Script (GAS)
- Agent Frameowork: LangChain on Vertex AI / Firebase Genkit
- LLM: Gemini 2.5 Flash
- Search Tool: Google Programmable Search Engine (CSE) to restrict search to academic domains (scholar.google.com, nih.gov, arxiv.org)
- Hallucination Killer: Vertex AI Grounding for force citation

In [5]:
import functions_framework
import vertexai
from vertexai.generative_models import GenerativeModel, Tool, GenerationConfig, FunctionDeclaration
from vertexai.generative_models import grounding
from google import genai
from google.genai import types
from google.genai.types import GenerateContentConfig, Tool, GoogleSearch
from flask import jsonify, Flask, request
from pyngrok import ngrok
from google.oauth2 import service_account
from kaggle_secrets import UserSecretsClient
import os
from google.auth import transport
import google.auth
import tempfile
import json

In [6]:
temp_path = None  # Global for cleanup
client = None  # Global client for Flask access
try:
    # Load secrets (Kaggle) or fallback to manual input (Colab)
    if UserSecretsClient:
        NGROK_AUTH_TOKEN = UserSecretsClient().get_secret("NGROK_AUTH_TOKEN")
        GOOGLE_PROJECT_ID = UserSecretsClient().get_secret("PROJECT_ID")
        SA_JSON_CONTENT = UserSecretsClient().get_secret("GCP_SA_JSON")
    else:
        # For local/Colab: Set via os.environ
        NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
        GOOGLE_PROJECT_ID = os.getenv("PROJECT_ID")
        SA_JSON_CONTENT = os.getenv("GCP_SA_JSON")

    if not all([NGROK_AUTH_TOKEN, GOOGLE_PROJECT_ID, SA_JSON_CONTENT]):
        raise ValueError("Missing required secrets/environment variables")

    # Set up ADC via service account JSON (as per 2025 docs)
    with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as temp_file:
        json.dump(json.loads(SA_JSON_CONTENT), temp_file)
        temp_path = temp_file.name

    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = temp_path

    # === CRITICAL: Set Vertex AI environment variables (fixes "Client not initialized") ===
    os.environ['GOOGLE_CLOUD_PROJECT'] = GOOGLE_PROJECT_ID
    os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'  # Or 'global'
    os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'  # Enables Vertex backend

    # Initialize client (simple default; routes to Vertex AI via env vars)
    client = genai.Client()  # No params needed—uses ADC + env vars per docs

    # Test client quickly (optional, for debugging)
    print("Testing client initialization...")
    test_response = client.models.generate_content(model="gemini-2.5-flash", contents="Test")
    print(f"Client test successful: {test_response.text[:50]}...")

    # Cleanup temp file early (ADC caches it)
    if temp_path:
        os.unlink(temp_path)
        temp_path = None
        print("Temp credentials file cleaned up.")

    # Setup ngrok
    ngrok.kill()
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    print("✅ Authentication and Google GenAI client initialized successfully (Vertex AI backend).")

except Exception as e:
    print("\n❌ AUTHENTICATION FAILED")
    print(f"Error: {e}")
    if temp_path:
        os.unlink(temp_path)
    raise

Testing client initialization...
Client test successful: Hello! I'm here and ready. How can I help you toda...
Temp credentials file cleaned up.
✅ Authentication and Google GenAI client initialized successfully (Vertex AI backend).              


In [7]:
google_search_tool = Tool(google_search=GoogleSearch())

In [8]:
model_name = "gemini-2.5-flash"

In [9]:
# --- STEP 3: HELPER FUNCTION FOR SAFE TEXT EXTRACTION (FIXES ATTRIBUTEERROR) ---
def extract_text_from_response(response):
    """Safely extract text from the first candidate, handling tool calls or empty parts (per docs)."""
    if not hasattr(response, 'candidates') or not response.candidates:
        return None
    candidate = response.candidates[0]
    if not hasattr(candidate, 'content') or not candidate.content.parts:
        return None
    part = candidate.content.parts[0]
    if hasattr(part, 'text') and part.text:
        return part.text.strip()
    elif hasattr(part, 'functionCall'):
        # Tool call detected (e.g., search grounding); no text, but grounding may have metadata
        print("Tool call detected (e.g., Google Search); processing metadata...")
        return None  # Trigger retry or use metadata
    else:
        # Safety block or empty
        print(f"No text; finish_reason: {getattr(candidate, 'finish_reason', 'Unknown')}")
        return None

In [10]:
def extract_grounding_metadata(response):
    """Extract sources from grounding_metadata (only if present post-tool)."""
    sources = []
    web_queries = []
    if hasattr(response, 'grounding_metadata') and response.grounding_metadata:
        metadata = response.grounding_metadata
        web_queries = getattr(metadata, 'web_search_queries', [])
        print(f"Search queries used: {web_queries}")  # Optional logging

        grounding_chunks = getattr(metadata, 'grounding_chunks', [])
        for chunk in grounding_chunks:
            if hasattr(chunk, 'web') and chunk.web:
                web = chunk.web
                sources.append({
                    "title": getattr(web, 'title', 'No title'),
                    "url": getattr(web, 'uri', ''),
                    "snippet": getattr(web, 'snippet', '')[:200] if hasattr(web, 'snippet') else ''
                })

        # Optional: Confidence scores
        grounding_supports = getattr(metadata, 'grounding_supports', [])
        for support in grounding_supports:
            if hasattr(support, 'confidence_scores'):
                print(f"Confidence for segment '{getattr(support.segment, 'text', '')[:50]}...': {support.confidence_scores}")

    return sources, web_queries

In [11]:
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

In [12]:
app = Flask(__name__)

In [13]:
@app.route('/validate', methods=['POST'])
def validate_research():
    global client  # Ensure access to module-level client
    if client is None:
        return jsonify({"error": "Client not initialized"}), 500

    try:
        data = request.get_json()
        if not data or 'text' not in data:
            return jsonify({"error": "Missing 'text' field in JSON"}), 400

        user_text = data['text'].strip()
        if not user_text:
            return jsonify({"error": "Text is empty"}), 400

        print(f"Fact-checking: {user_text[:100]}...")

        # Initial prompt
        prompt = f"""
        Carefully analyze the following statement for factual accuracy using Google Search grounding.

        Statement to verify:
        "{user_text}"

        Instructions:
        - Use search results to verify every factual claim.
        - If the statement is accurate and well-supported → respond with the original text followed by [VERIFIED] and list sources.
        - If any part is false, misleading, or unsupported → respond with [FALSE CLAIM DETECTED], explain the error, and provide the correct information with sources.
        - Always cite sources clearly. Provide a direct response after searching.
        """

        max_retries = 1
        response = None
        for attempt in range(max_retries + 1):
            response = client.models.generate_content(
                model=model_name,
                contents=[prompt],  # List format per docs
                config=GenerateContentConfig(
                    tools=[google_search_tool],  # Enables grounding
                    temperature=1.0,  # Recommended for grounded results
                    max_output_tokens=8192
                )
            )

            text = extract_text_from_response(response)
            if text:
                break  # Success: text extracted
            elif attempt < max_retries:
                print(f"Retry {attempt + 1}: No text (tool call); refining prompt...")
                prompt += "\nAfter searching, provide a complete analysis without further tool calls."  # Encourage final response
            else:
                # Fallback: Use empty text or error
                text = "Analysis unavailable due to tool processing error. Please retry."

        # Extract metadata (works even on tool calls if grounding occurred)
        sources, web_queries = extract_grounding_metadata(response)

        result = {
            "analysis": text,
            "suggested_citations": sources,
            "is_hallucination": "FALSE CLAIM DETECTED" in text.upper() if text else False,
            "verified": "[VERIFIED]" in text if text else False,
            "search_queries_used": web_queries  # For UI suggestions
        }

        return jsonify(result)

    except Exception as e:
        print(f"Generation error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({"error": "Model inference failed", "details": str(e)}), 500

In [15]:
# --- STEP 4: START SERVER ---
if __name__ == "__main__":
    public_url = ngrok.connect(5000, bind_tls=True).public_url
    print("\n🚀 YOUR LIVE VALIDATOR ENDPOINT IS READY!")
    print(f"   POST → {public_url}/validate")
    print(f"   Example payload: {{ \"text\": \"Water boils at 90°C at sea level\" }}")
    print("\n   → Copy this URL into your Google Apps Script!\n")

    # Run Flask with proper host/port
    app.run(host="0.0.0.0", port=5000)


🚀 YOUR LIVE VALIDATOR ENDPOINT IS READY!
   POST → https://undeprecatively-unpunctuated-kara.ngrok-free.dev/validate
   Example payload: { "text": "Water boils at 90°C at sea level" }

   → Copy this URL into your Google Apps Script!

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.19.2.2:5000
INFO:werkzeug:Press CTRL+C to quit


Fact-checking: analysis. Lancet Gastroenterol Hepatol 2023;8:553-564....


INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 13:59:08] "POST /validate HTTP/1.1" 200 -


Fact-checking: Malfertheiner P, Megraud F, Rokkas T, et al. Management of Helicobacter pylori infection: the Maastr...


...

INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:54] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:54] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:54] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:55] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:55] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:55] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:45:56] "POST /validate HTTP/1.1" 200 -
INFO

Fact-checking: Li RJ, Dai YY, Qin C, et al. Application of traditional Chinese medicine in treatment of Helicobacte...
Fact-checking: Iwata H. Application of in Silico Technologies for Drug Target Discovery and Pharmacokinetic Analysi...
Fact-checking: Zhang Y, Luo M, Wu P, et al. Application of Computational Biology and Artificial Intelligence in Dru...
Fact-checking: Pettersen EF, Goddard TD, Huang CC, et al. UCSF Chimera--a visualization system for exploratory rese...
Fact-checking: Trott O, Olson AJ. AutoDock Vina: improving the speed and accuracy of docking with a new scoring fun...
Fact-checking: Wang S, Hou Y, Li X, et al. Practical Implementation of Artificial Intelligence-Based Deep Learning ...
Fact-checking: Karimi M, Wu D, Wang Z, et al. DeepAffinity: interpretable deep learning of compound-protein affinit...
Fact-checking: Yang K, Xie Z, Li Z, et al. MolProphet: A One-Stop, General Purpose, and AI-Based Platform for the E...
Fact-checking: Cheon C. Synergistic effects of h

INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:22] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:23] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:24] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:24] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:24] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:24] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:24] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:25] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:25] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:25] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:26] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:27] "POST /validate HTTP/1.1" 200 -
INFO

Fact-checking: Qian HS, Li WJ, Dang YN, et al. Ten-Day Vonoprazan-Amoxicillin Dual Therapy as a First-Line Treatmen...
Fact-checking: Hu J, Mei H, Su NY, et al. Eradication rates of Helicobacter pylori in treatment-naive patients foll...
Fact-checking: Smith, J. A., & Rodriguez, K. L. (2023). Quantum entanglement in everyday household appliances: Evid...


INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:58] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:46:59] "POST /validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Nov/2025 15:47:07] "POST /validate HTTP/1.1" 200 -
